In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from IPython.display import clear_output

In [2]:
def load_image(path):
  """Loads an image from hard drive using the given path.
  """
  return cv2.imread(path)

In [3]:
def show_image(image):
  """Shows the image.
  """
  plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [4]:
def convert_to_grayscale(image):
    """Converts the given image to the greyscale.
    """
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [5]:
def __fast_convolve(image, kernel):
    """Applies kernel to the given image.
    """
    image_fft = np.fft.rfft2(image)
    kernel_fft = np.fft.rfft2(kernel,image.shape)
    return np.fft.irfft2(image_fft*kernel_fft)

def get_edges(image):
    """Finds image edges.
    """
    sx = np.array([[ 1, 2, 1],
                   [ 0, 0, 0],
                   [-1,-2,-1]])
    
    sy = np.array([[ 1, 0, -1],
                   [ 2, 0, -2],
                   [ 1, 0, -1]])
    
    edge_horizontal = __fast_convolve(image, sx)
    edge_vertical = __fast_convolve(image, sy)
    
    return np.sqrt(np.square(edge_horizontal) + np.square(edge_vertical))

In [6]:
def calculate_cost(image_edges):
    """Calculates energy/cost of the edges.
       In other words, this function finds areas with interesting
       objects in it.
    """
    r, c = image_edges.shape
    cost = np.zeros(image_edges.shape)
    cost[r-1,:] = image_edges[r-1,:]
    
    for i in range(r-2,-1,-1):
        for j in range(c):
            c1,c2 = max(j-1,0),min(c,j+2)
            cost[i][j] = image_edges[i][j] + cost[i+1,c1:c2].min()
                
    return cost

In [7]:
def find_seam(image_cost):
    """Finds seam (path) with the minimum energy/cost.
    """
    r, c = image_cost.shape
    
    seam = []
    j = image_cost[0].argmin()
    seam.append(j)
    
    for i in range(r-1):
        c1,c2 = max(j-1,0),min(c,j+2)
        j = max(j-1,0)+image_cost[i+1,c1:c2].argmin()
        seam.append(j)

    return seam

In [8]:
def remove_seam(image, seam):
    """Removes the given seam from the image.
    """
    r, c, _ = image.shape
    new_image = np.zeros((r,c,3))

    for i, j in enumerate(seam):
        new_image[i,0:j,:] = image[i,0:j,:]
        new_image[i,j:c-1,:] = image[i,j+1:c,:]
    return new_image[:,:-1,:].astype(np.uint8)

In [9]:
def draw_seam(image, seam):
    """Draws the given seam on the given image.
       Comes handy for debugging seams.
    """
    image_canvas = np.copy(image)
    image_canvas[range(len(seam)),seam,:] = np.array([255,0,0])
    return image_canvas

In [10]:
def horizontal_seam_carving(image, dw):
    """Horizontal content-aware image resizing.
       Horizontal means that the methods removes columns of the image.
    """
    carved_image = np.copy(image) 
    for i in range(0, dw):
        carved_image = horizontal_seam_carving_one_step(carved_image)
        clear_output(wait=True)
        print("[Seam Carving] Progress: {:.0f}%".format((i + 1) / dw * 100))
    return carved_image

In [11]:
def vertical_seam_carving(image, dh):
    """Vertical content-aware image resizing.
       Vertical means that the methods removes rows of the image.
    """
    carved_image = np.copy(image)
    carved_image = carved_image.transpose(1, 0, 2)
    carved_image = horizontal_seam_carving(carved_image, dh)
    carved_image = carved_image.transpose(1, 0, 2)
    return carved_image

**Exercise:** you need to implement one step of seam carving algorithm. In other words, we need to remove only 1 vertical column from the image.

Remember the steps that we discussed during the lecture:
1. Convert image to grayscale
2. Find edges
3. Calculate energy function (aka cost function)
4. Find a seam (aka path from top to bottom) with the minimal energy
5. Remove the seam from the image
6. Return the image

**Note:** we prepared a few handy methods for you.

- `convert_to_grayscale(img)`: converts a bgr image to grayscale
- `get_edges(grayscale_image)`: extracts edges from a grayscale image
- `calculate_cost(edges)`: calculates energy (aka cost) matrix
- `find_seam(cost_matrix)`: find a seam with min energy
- `remove_seam(img)`: removes a seam from the image

- `draw_seam(img)`: draws a seam on the image (instead of removing it), may come handy during debugging

- `vertical_seam_carving(img, dh)`: carves `dh` rows from the `img`
- `horizontal_seam_carving(img, dw)`: carves `dw` columns from the `img`


**Test:** call a method with an image and you should see a resized version of the image.

In [ ]:
def horizontal_seam_carving_one_step(image):
    """Implement one step of seam carving algorithm."""
    pass

In [ ]:
show_image(load_image("images/plane_4.jpeg"))

In [ ]:
%%time
# This is a function to test your code. Replace the image with any image you want.
show_image(vertical_seam_carving(load_image("images/plane_4.jpeg"), 200))